In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
from nltk.corpus import wordnet
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
TA_df = pd.read_csv("data/TripAdvisor_data_cleaned.csv")
GG_df = pd.read_csv("data/GoogleReview_data_cleaned.csv")

In [4]:
TA_df.head()

,Author,Title,Review,Rating,Dates,Restaurant,Location
0,kmc1e2018,Chambers Bar and Grill,"David, Thanga, Mikail and Chef Steven gave bri...",5.0,Reviewed yesterday,Chambers Grill,KL
1,"MizOthmanKuala Lumpur, Malaysia",Always a pleasure. Place to celebrate.,We visited for family celebration and as usual...,5.0,Reviewed 5 days ago,Chambers Grill,KL
2,"Relax36268533224Hjarno, Denmark",Enjoying good service and good food after a lo...,Perfect dinner after a long journey.After trav...,5.0,Reviewed 1 week ago,Chambers Grill,KL
3,tstrry,Great Steak for Couples,Had a great Tomahawk for me and hubby thanks t...,5.0,Reviewed 1 week ago,Chambers Grill,KL
4,Naim_123456789,Dining,"Excellent ambient. Excellent service by David,...",5.0,Reviewed 1 week ago,Chambers Grill,KL


In [5]:
GG_df.head()

,Author,Rating,Review,Restaurant,Location
0,Jia Pin Lee,4.0,Came here for the High Tea. Great service espe...,Cuisines Restaurant,Ipoh
1,Chui Yi Lum,2.0,"5 stars for the service, even though some of t...",Cuisines Restaurant,Ipoh
2,liezel wong,1.0,"Hi, thank you for your service. But! i feel so...",Cuisines Restaurant,Ipoh
3,Nazri Nor,1.0,I have the worse buffer dinner ever so far. Th...,Cuisines Restaurant,Ipoh
4,Fakru Imran's Channel,5.0,"That's are Known 5 Elmark "" 9H72 "" & KDK "" 3 K...",Cuisines Restaurant,Ipoh


In [6]:
TA_df.drop(columns=['Author', 'Dates', 'Title'], inplace=True)
GG_df.drop(columns=['Author'], inplace=True)

In [7]:
df = pd.concat([TA_df, GG_df], ignore_index=True)

In [8]:
df.head()

,Review,Rating,Restaurant,Location
0,"David, Thanga, Mikail and Chef Steven gave bri...",5.0,Chambers Grill,KL
1,We visited for family celebration and as usual...,5.0,Chambers Grill,KL
2,Perfect dinner after a long journey.After trav...,5.0,Chambers Grill,KL
3,Had a great Tomahawk for me and hubby thanks t...,5.0,Chambers Grill,KL
4,"Excellent ambient. Excellent service by David,...",5.0,Chambers Grill,KL


In [9]:
df.shape

(361784, 4)

In [10]:
print('Total unique restaurants: '+str(len(df['Restaurant'].unique())))
print('Total unique location: '+str(len(df['Location'].unique())))


Total unique restaurants: 3515
Total unique location: 10


In [11]:
def best_rated():
    average_rating = df.groupby('Restaurant')['Rating'].agg(['mean', 'count']).reset_index()
    average_rating.columns = ['Restaurant', 'Mean', 'Num']
    average_rating = average_rating.sort_values(by=['Mean', 'Num'], ascending=False)
    return average_rating

best_rated()

,Restaurant,Mean,Num
2183,Potters Place Cafe,5.0,38
2173,Pool Bar & Grill,5.0,34
386,Busdriver.Cafe,5.0,20
1504,Le Shady Trinity Cafe,5.0,20
169,Aryan Restaurant,5.0,18
...,...,...,...
2112,Penang Road Famous Teochew Chendul,1.0,1
2244,Raw - Brew & Dine,1.0,1
2967,Ten Here Cafe,1.0,1
3339,Warung Plus Plus,1.0,1
